1. Monthly Cost and Utilization Trends by Specialty

In [0]:
SELECT 
    DATE_TRUNC('month', c.claim_date) AS claim_month,
    p.specialty,
    COUNT(*) AS total_claims,
    SUM(c.paid_amount::FLOAT) AS total_paid,
    AVG(c.claim_amount::FLOAT) AS avg_claim
FROM hcp.claims_data c
JOIN hcp.providers_data p ON c.provider_id = p.provider_id
GROUP BY claim_month, p.specialty
ORDER BY claim_month DESC, total_paid DESC;

2. Top Performing Providers by Approval Rate

In [0]:
SELECT 
    p.provider_id,
    p.name,
    p.specialty,
    COUNT(*) AS total_claims,
    SUM(CASE WHEN c.claim_status = 'Approved' THEN 1 ELSE 0 END) AS approved_claims,
    ROUND(100.0 * SUM(CASE WHEN c.claim_status = 'Approved' THEN 1 ELSE 0 END) / COUNT(*), 2) AS approval_rate
FROM hcp.claims_data c
JOIN hcp.providers_data p ON c.provider_id = p.provider_id
GROUP BY p.provider_id, p.name, p.specialty
HAVING COUNT(*) > 20
ORDER BY approval_rate DESC
LIMIT 10;

3. High-Risk providers Based on Claim Volumn and Risk Score

In [0]:
SELECT 
    p.provider_id,
    p.name,
    COUNT(c.claim_id) AS total_claims,
    ROUND(AVG(p.risk_score), 2) AS avg_risk_score
FROM hcp.providers_data p
JOIN hcp.claims_data c ON p.provider_id = c.provider_id
GROUP BY p.provider_id, p.name
HAVING COUNT(*) > 50 AND AVG(p.risk_score) > 0.3
ORDER BY avg_risk_score DESC;

4.  Outlier Detection : Providers With Unsually High Claim Amounts

In [0]:
SELECT 
    provider_id,
    AVG(claim_amount::FLOAT) AS avg_claim,
    MAX(claim_amount::FLOAT) AS max_claim
FROM hcp.claims_data
GROUP BY provider_id
HAVING MAX(claim_amount::FLOAT) > 2 * AVG(claim_amount::FLOAT)
ORDER BY max_claim DESC;

5. Emergency Service Utilization by Region

In [0]:
SELECT 
    location,
    COUNT(*) AS emergency_visits,
    ROUND(100.0 * SUM(CASE WHEN is_emergency THEN 1 ELSE 0 END) / COUNT(*), 2) AS emergency_percentage
FROM 
    hcp.claims_data
GROUP BY 
    location
ORDER BY 
    emergency_percentage DESC;

6. New vs Returning Patient Behavior by Month

In [0]:
WITH ranked_claims AS (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY patient_id ORDER BY claim_date) AS visit_number
    FROM hcp.claims_data
)
SELECT 
    DATE_TRUNC('month', claim_date) AS month,
    SUM(CASE WHEN visit_number = 1 THEN 1 ELSE 0 END) AS new_patients,
    SUM(CASE WHEN visit_number > 1 THEN 1 ELSE 0 END) AS returning_patients
FROM ranked_claims
GROUP BY month
ORDER BY month DESC;

7. Specialty-wise Revenue and Provider Experience Correlation

In [0]:
SELECT 
    p.specialty,
    COUNT(c.claim_id) AS total_claims,
    ROUND(AVG(c.paid_amount::FLOAT), 2) AS avg_paid,
    ROUND(AVG(p.years_of_experience), 1) AS avg_experience
FROM hcp.claims_data c
JOIN hcp.providers_data p ON c.provider_id = p.provider_id
GROUP BY p.specialty
ORDER BY total_claims DESC;

8. State-wise Acceptance of New Patients

In [0]:
SELECT 
    state,
    SUM(CASE WHEN accepting_new_patients THEN 1 ELSE 0 END) AS accepting_count,
    COUNT(*) AS total_providers,
    ROUND(100.0 * SUM(CASE WHEN accepting_new_patients THEN 1 ELSE 0 END) / COUNT(*), 2) AS acceptance_rate
FROM 
    hcp.providers_data
GROUP BY 
    state
ORDER BY 
    acceptance_rate ASC;

9. Procedure Code Utilization Trend

In [0]:
SELECT 
    procedure_code,
    DATE_TRUNC('month', claim_date) AS month,
    COUNT(*) AS usage_count,
    ROUND(SUM(paid_amount::FLOAT), 2) AS total_paid
FROM hcp.claims_data
GROUP BY procedure_code, month
HAVING COUNT(*) > 50
ORDER BY month DESC, usage_count DESC;

10. Patient Condition Heatmap by Diagnosis Code

In [0]:
SELECT 
    d.state,
    c.diagnosis_code,
    COUNT(*) AS case_count
FROM hcp.claims_data c
JOIN hcp.patients_data d ON c.patient_id = d.patient_id
GROUP BY d.state, c.diagnosis_code
HAVING COUNT(*) > 30
ORDER BY case_count DESC;